# NSE Equity and Derivatives Analysis

Analyze NSE equity bhavcopy, security deliverable, and derivatives reports from a local data repository. Configure the path and choose a symbol to see charts and export reports.

## 1) Set Notebook Parameters (Data Path, Date Range, Symbol)

Edit the path, dates, and default symbol as needed. Paths with spaces are handled safely.

In [97]:
# Parameters
from pathlib import Path
from datetime import date, timedelta

# Resolve repo root (this notebook lives in repo/notebooks)
REPO_ROOT = Path.cwd().parent
DATA_LINK = REPO_ROOT / "data"
# Base folder now via repo-local data link -> OneDrive data folder
DATA_ROOT = DATA_LINK / "Full Bhavcopy and Security deliverable"

# Restrict scanning to only these subfolders
ALLOWED_DIRS = [
    "202508",
    "202509",
    "202510",
]
# Build explicit list of directories to scan
DATA_DIRS = [DATA_ROOT / d for d in ALLOWED_DIRS]

REPORTS_DIR = Path("notebooks") / "reports"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# Default analysis window: last 90 days (adjust as your dataset range)
end_date = date.today()
start_date = end_date - timedelta(days=90)

# Default symbol (uppercase). Change as desired, or use the interactive widget later.
default_symbol = "RELIANCE"

print("REPO_ROOT:", REPO_ROOT)
print("DATA_LINK:", DATA_LINK, "exists:", DATA_LINK.exists())
print("DATA_ROOT:", DATA_ROOT, "exists:", DATA_ROOT.exists())
print("Scanning only subfolders (existence):")
existing_dirs = 0
for d in DATA_DIRS:
    ex = d.exists()
    if ex:
        existing_dirs += 1
    print(" -", d, "exists:" , ex)
if existing_dirs == 0:
    print("WARNING: None of the ALLOWED_DIRS exist under DATA_ROOT. Discovery will fall back to scanning the entire DATA_ROOT.")
print("Date range:", start_date, "to", end_date)
print("Default symbol:", default_symbol)

REPO_ROOT: e:\workspace\nseoptions
DATA_LINK: e:\workspace\nseoptions\data exists: True
DATA_ROOT: e:\workspace\nseoptions\data\Full Bhavcopy and Security deliverable exists: True
Scanning only subfolders (existence):
 - e:\workspace\nseoptions\data\Full Bhavcopy and Security deliverable\202508 exists: True
 - e:\workspace\nseoptions\data\Full Bhavcopy and Security deliverable\202509 exists: True
 - e:\workspace\nseoptions\data\Full Bhavcopy and Security deliverable\202510 exists: True
Date range: 2025-08-01 to 2025-10-30
Default symbol: RELIANCE


In [98]:
# Helper: Create/repair repo-local data junction (Windows)
from pathlib import Path
import platform, subprocess

# If REPO_ROOT/DATA_LINK are not defined (cell re-run out of order), reconstruct them
try:
    REPO_ROOT
except NameError:
    REPO_ROOT = Path.cwd().parent
DATA_LINK = REPO_ROOT / "data"
# Change this target if your data folder lives elsewhere
DATA_TARGET = Path(r"C:\Users\atulk_0o0fet8\OneDrive\Finance\AnalysisStock\Data")

print("Data link check:")
print("  REPO_ROOT:", REPO_ROOT)
print("  DATA_LINK:", DATA_LINK)
print("  DATA_TARGET:", DATA_TARGET)

if platform.system() != "Windows":
    print("Non-Windows OS detected; skipping junction creation.")
else:
    ps_script = f"""
$ErrorActionPreference = 'Stop';
$Link = [System.IO.Path]::GetFullPath('{str(DATA_LINK)}');
$Target = [System.IO.Path]::GetFullPath('{str(DATA_TARGET)}');
if (Test-Path -LiteralPath $Link) {{
  try {{
    $item = Get-Item -LiteralPath $Link -Force
    $isJunction = ($item.Attributes -band [IO.FileAttributes]::ReparsePoint) -ne 0
    $lt = $null
    try {{ $lt = $item.LinkTarget }} catch {{}}
    if ($isJunction -and ($lt -eq $Target)) {{
      Write-Output "OK: Junction already correct -> $lt"
    }} else {{
      Remove-Item -LiteralPath $Link -Force -Recurse
      New-Item -ItemType Junction -Path $Link -Target $Target | Out-Null
      Write-Output "CREATED: $Link -> $Target"
    }}
  }} catch {{
    Write-Output "ERROR: $($_.Exception.Message)"
    exit 1
  }}
}} else {{
  New-Item -ItemType Junction -Path $Link -Target $Target | Out-Null
  Write-Output "CREATED: $Link -> $Target"
}}
"""
    try:
        res = subprocess.run(["pwsh", "-NoProfile", "-Command", ps_script], capture_output=True, text=True)
        if res.stdout:
            print(res.stdout.strip())
        if res.returncode != 0:
            print("PowerShell error:", res.stderr.strip())
    except FileNotFoundError:
        # Fallback to cmd mklink
        try:
            subprocess.run(["cmd", "/c", "rmdir", "/s", "/q", str(DATA_LINK)], check=False)
            subprocess.run(["cmd", "/c", "mklink", "/J", str(DATA_LINK), str(DATA_TARGET)], check=True)
            print(f"CREATED (cmd mklink): {DATA_LINK} -> {DATA_TARGET}")
        except Exception as e:
            print("Failed to create junction via cmd mklink:", e)

# Show final existence for quick sanity
print("Exists DATA_LINK:", DATA_LINK.exists())
print("Exists DATA_ROOT folder:", (DATA_LINK / "Full Bhavcopy and Security deliverable").exists())

Data link check:
  REPO_ROOT: e:\workspace\nseoptions
  DATA_LINK: e:\workspace\nseoptions\data
  DATA_TARGET: C:\Users\atulk_0o0fet8\OneDrive\Finance\AnalysisStock\Data
OK: Junction already correct -> C:\Users\atulk_0o0fet8\OneDrive\Finance\AnalysisStock\Data
Exists DATA_LINK: True
Exists DATA_ROOT folder: True
OK: Junction already correct -> C:\Users\atulk_0o0fet8\OneDrive\Finance\AnalysisStock\Data
Exists DATA_LINK: True
Exists DATA_ROOT folder: True


## 2) Install and Import Libraries

In [99]:
# If running in a fresh kernel, you may need to install dependencies. Uncomment to install.
# %pip install pandas numpy pyarrow plotly ipywidgets kaleido --quiet

import os, re, glob, math, json, hashlib, warnings, zipfile
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Enable inline rendering in VS Code notebooks
import plotly.io as pio
pio.renderers.default = "notebook_connected"  # fallback to 'vscode' or 'notebook'

# Widgets (optional interactive controls)
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    WIDGETS_AVAILABLE = True
except Exception:
    WIDGETS_AVAILABLE = False

print("Libraries ready. Widgets:", WIDGETS_AVAILABLE)

Libraries ready. Widgets: True


## 3) Discover and Index Raw Files

In [100]:
def infer_date_from_name(name: str):
    # Try patterns like CMDDMMMYYYYbhav.csv, yyyyMMdd, ddMMyyyy, etc.
    # Adjust to your actual naming if different.
    candidates = []
    # 1) yyyyMMdd
    m = re.search(r"(?P<y>20\d{2})(?P<m>0[1-9]|1[0-2])(?P<d>[0-3]\d)", name)
    if m:
        try:
            candidates.append(datetime(int(m['y']), int(m['m']), int(m['d'])).date())
        except Exception:
            pass
    # 2) ddMMyyyy
    m = re.search(r"(?P<d>[0-3]\d)(?P<m>0[1-9]|1[0-2])(?P<y>20\d{2})", name)
    if m:
        try:
            candidates.append(datetime(int(m['y']), int(m['m']), int(m['d'])).date())
        except Exception:
            pass
    # 3) dd-MMM-YYYY (e.g., 31-OCT-2025)
    m = re.search(r"([0-3]?\d)-([A-Za-z]{3})-(20\d{2})", name)
    if m:
        try:
            candidates.append(datetime.strptime(m.group(0), "%d-%b-%Y").date())
        except Exception:
            pass
    return min(candidates) if candidates else None

# Broaden file patterns to include common uppercase extensions
FILE_PATTERNS = [
    "**/*.csv", "**/*.CSV",
    "**/*.csv.gz", "**/*.CSV.GZ",
    "**/*.zip", "**/*.ZIP",
]

# Determine where to search: existing restricted list else fallback to DATA_ROOT
SEARCH_DIRS = []
try:
    tmp = []
    for d in DATA_DIRS:
        p = Path(d)
        if p.exists():
            tmp.append(p)
    if tmp:
        SEARCH_DIRS = tmp
    else:
        SEARCH_DIRS = [Path(DATA_ROOT)]
except NameError:
    SEARCH_DIRS = [Path(DATA_ROOT)]

index_rows = []
for base in SEARCH_DIRS:
    for pat in FILE_PATTERNS:
        for p in base.glob(pat):
            name = p.name
            lower = name.lower()
            tokens = [t for t in re.split(r'[^a-z0-9]+', lower) if t]
            # classify using token presence (underscores are not word boundaries for \b)
            ftype = None
            if any(t in tokens for t in ['fo','deriv','fut','opt','oi']):
                ftype = 'derivatives'
            elif ('deliver' in lower) or ('security' in lower and ('deliv' in lower or 'delivery' in lower)):
                ftype = 'deliverable'
            elif ('bhav' in lower) or ('bhavcopy' in lower) or any(t in tokens for t in ['cm','eq','equity']):
                ftype = 'equity_bhav'

            fdate = infer_date_from_name(name)
            try:
                size = p.stat().st_size
            except Exception:
                size = None
            index_rows.append({
                'file_path': str(p),
                'file_type': ftype,
                'file_date': fdate,
                'size': size,
            })

files_index = pd.DataFrame(index_rows)
print("Search bases:", [str(b) for b in SEARCH_DIRS])
print("Indexed files:", len(files_index))
if len(files_index) == 0:
    print("WARNING: No files found. Verify DATA_ROOT and ALLOWED_DIRS or remove the restriction.")
print(files_index.groupby('file_type')['file_path'].count().to_dict())
files_index.head(10)

Search bases: ['e:\\workspace\\nseoptions\\data\\Full Bhavcopy and Security deliverable\\202508', 'e:\\workspace\\nseoptions\\data\\Full Bhavcopy and Security deliverable\\202509', 'e:\\workspace\\nseoptions\\data\\Full Bhavcopy and Security deliverable\\202510']
Indexed files: 244
{'derivatives': 122, 'equity_bhav': 122}


,file_path,file_type,file_date,size
0,e:\workspace\nseoptions\data\Full Bhavcopy and...,equity_bhav,2025-08-01,527623
1,e:\workspace\nseoptions\data\Full Bhavcopy and...,equity_bhav,2025-08-04,537795
2,e:\workspace\nseoptions\data\Full Bhavcopy and...,equity_bhav,2025-08-05,531720
3,e:\workspace\nseoptions\data\Full Bhavcopy and...,equity_bhav,2025-08-06,533911
4,e:\workspace\nseoptions\data\Full Bhavcopy and...,equity_bhav,2025-08-07,532106
5,e:\workspace\nseoptions\data\Full Bhavcopy and...,equity_bhav,2025-08-08,527785
6,e:\workspace\nseoptions\data\Full Bhavcopy and...,equity_bhav,2025-08-11,536748
7,e:\workspace\nseoptions\data\Full Bhavcopy and...,equity_bhav,2025-08-12,531597
8,e:\workspace\nseoptions\data\Full Bhavcopy and...,equity_bhav,2025-08-13,531669
9,e:\workspace\nseoptions\data\Full Bhavcopy and...,equity_bhav,2025-08-14,528188


## 4) Load Equity Bhavcopy Files (Batch)

In [101]:
# DEBUG: Inspect one discovered equity file to understand its structure
sample_eq = files_index[files_index['file_type']=='equity_bhav'].head(1)
if not sample_eq.empty:
    fp = sample_eq.iloc[0]['file_path']
    print("Sample equity file:", fp)
    df0 = pd.read_csv(fp, compression='infer', low_memory=False)
    print("read_csv infer shape:", df0.shape)
    print("columns:", list(df0.columns)[:40])
    display(df0.head(3))

    # simulate mapping used by read_equity_bhav
    cols = {str(c).strip(): str(c).strip() for c in df0.columns}
    U = {k.upper(): v for k, v in cols.items()}
    def pick(col):
        aliases = {
            'SYMBOL': ['SYMBOL', 'SYMB', 'TICKER', 'TckrSymb'],
            'SERIES': ['SERIES', 'SctySrs'],
            'OPEN': ['OPEN', 'OPEN_PRICE', 'OpnPric'],
            'HIGH': ['HIGH', 'HIGH_PRICE', 'HghPric'],
            'LOW': ['LOW', 'LOW_PRICE', 'LwPric'],
            'CLOSE': ['CLOSE', 'CLOSE_PRICE', 'LAST', 'ClsPric', 'LastPric'],
            'TOTTRDQTY': ['TOTTRDQTY', 'VOLUME', 'TOTALTRDQTY', 'TradedQty', 'TtlTradgVol'],
            'TOTTRDVAL': ['TOTTRDVAL', 'TURNOVER', 'TOTALTRDVAL', 'TtlTrfVal', 'TtlTradedVal'],
            'TOTALTRADES': ['TOTALTRADES', 'TRADES', 'TotTrdsExctd'],
            'TIMESTAMP': ['TIMESTAMP', 'DATE', 'TradDt', 'BizDt']
        }
        for a in aliases.get(col, [col]):
            if a in cols:
                return cols[a]
            au = a.upper()
            if au in U:
                return U[au]
        return None
    picks = {k: pick(k) for k in ['SYMBOL','SERIES','OPEN','HIGH','LOW','CLOSE','TOTTRDQTY','TOTTRDVAL','TOTALTRADES','TIMESTAMP']}
    print("picks:", picks)
    out = pd.DataFrame()
    for k, src in picks.items():
        if src is not None and src in df0.columns:
            out[k] = df0[src]
        else:
            out[k] = np.nan
    out['TIMESTAMP'] = pd.to_datetime(out['TIMESTAMP'], errors='coerce')
    print("non-null counts:")
    print(out[['SYMBOL','TIMESTAMP']].notna().sum())
    display(out.head(3))
else:
    print("No equity files to inspect.")

Sample equity file: e:\workspace\nseoptions\data\Full Bhavcopy and Security deliverable\202508\BhavCopy_NSE_CM_0_0_0_20250801_F_0000.csv
read_csv infer shape: (3055, 34)
columns: ['TradDt', 'BizDt', 'Sgmt', 'Src', 'FinInstrmTp', 'FinInstrmId', 'ISIN', 'TckrSymb', 'SctySrs', 'XpryDt', 'FininstrmActlXpryDt', 'StrkPric', 'OptnTp', 'FinInstrmNm', 'OpnPric', 'HghPric', 'LwPric', 'ClsPric', 'LastPric', 'PrvsClsgPric', 'UndrlygPric', 'SttlmPric', 'OpnIntrst', 'ChngInOpnIntrst', 'TtlTradgVol', 'TtlTrfVal', 'TtlNbOfTxsExctd', 'SsnId', 'NewBrdLotQty', 'Rmks', 'Rsvd1', 'Rsvd2', 'Rsvd3', 'Rsvd4']


,TradDt,BizDt,Sgmt,Src,FinInstrmTp,FinInstrmId,ISIN,TckrSymb,SctySrs,XpryDt,...,TtlTradgVol,TtlTrfVal,TtlNbOfTxsExctd,SsnId,NewBrdLotQty,Rmks,Rsvd1,Rsvd2,Rsvd3,Rsvd4
0,2025-08-01,2025-08-01,CM,NSE,STK,19078,IN0020200104,SGBJUN28,GB,NaN,...,420,4175051.35,41,F1,1,NaN,NaN,NaN,NaN,NaN
1,2025-08-01,2025-08-01,CM,NSE,STK,1342,IN0020200286,SGBN28VIII,GB,NaN,...,31,309284.00,21,F1,1,NaN,NaN,NaN,NaN,NaN
2,2025-08-01,2025-08-01,CM,NSE,STK,3742,IN0020210053,SGBMAY29I,GB,NaN,...,220,2183268.70,53,F1,1,NaN,NaN,NaN,NaN,NaN


picks: {'SYMBOL': 'TckrSymb', 'SERIES': 'SctySrs', 'OPEN': 'OpnPric', 'HIGH': 'HghPric', 'LOW': 'LwPric', 'CLOSE': 'ClsPric', 'TOTTRDQTY': 'TtlTradgVol', 'TOTTRDVAL': 'TtlTrfVal', 'TOTALTRADES': None, 'TIMESTAMP': 'TradDt'}
non-null counts:
SYMBOL       3055
TIMESTAMP    3055
dtype: int64


,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,TOTTRDQTY,TOTTRDVAL,TOTALTRADES,TIMESTAMP
0,SGBJUN28,GB,9863.00,10099.99,9860.0,9939.00,420,4175051.35,NaN,2025-08-01
1,SGBN28VIII,GB,9950.00,9999.00,9925.0,9979.50,31,309284.00,NaN,2025-08-01
2,SGBMAY29I,GB,9929.99,9938.00,9902.0,9932.43,220,2183268.70,NaN,2025-08-01


In [102]:
def read_equity_bhav(files_df: pd.DataFrame) -> pd.DataFrame:
    eq_files = files_df[files_df['file_type'] == 'equity_bhav']
    if eq_files.empty:
        print("No equity bhavcopy files found.")
        return pd.DataFrame()

    def read_csv_robust(fp: str) -> pd.DataFrame:
        errs = []
        tries = [
            dict(compression='infer', low_memory=False),
            dict(compression='infer', on_bad_lines='skip', low_memory=False),
            dict(compression='infer', engine='python', sep=None, on_bad_lines='skip', low_memory=False),
            dict(compression='infer', encoding='latin1', on_bad_lines='skip', low_memory=False),
        ]
        for kw in tries:
            try:
                return pd.read_csv(fp, **kw)
            except Exception as e:
                errs.append(f"{type(e).__name__}: {e}")
        # explicit handling for .zip with inner csv
        if str(fp).lower().endswith('.zip'):
            try:
                with zipfile.ZipFile(fp) as z:
                    members = [n for n in z.namelist() if n.lower().endswith(('.csv','.txt'))]
                    if members:
                        with z.open(members[0]) as f:
                            for kw in [
                                dict(low_memory=False),
                                dict(on_bad_lines='skip', low_memory=False),
                                dict(engine='python', sep=None, on_bad_lines='skip', low_memory=False),
                                dict(encoding='latin1', on_bad_lines='skip', low_memory=False),
                            ]:
                                try:
                                    return pd.read_csv(f, **kw)
                                except Exception as e:
                                    errs.append(f"zip/{members[0]} {type(e).__name__}: {e}")
            except Exception as e:
                errs.append(f"ZipError: {e}")
        raise RuntimeError("; ".join(errs[-3:]))

    rows = []
    ok, fail = 0, 0
    debug_prints = 0
    for _, r in eq_files.sort_values('file_date', na_position='last').iterrows():
        fp = r['file_path']
        try:
            df = read_csv_robust(fp)
        except Exception as e:
            warnings.warn(f"Failed to read {fp}: {e}")
            fail += 1
            continue

        # Normalize expected columns (support NSE MII style names too)
        cols = {str(c).strip(): str(c).strip() for c in df.columns}
        U = {k.upper(): v for k, v in cols.items()}
        def get(col):
            # handle aliases
            aliases = {
                'SYMBOL': ['SYMBOL', 'SYMB', 'TICKER', 'TckrSymb'],
                'SERIES': ['SERIES', 'SctySrs'],
                'OPEN': ['OPEN', 'OPEN_PRICE', 'OpnPric'],
                'HIGH': ['HIGH', 'HIGH_PRICE', 'HghPric'],
                'LOW': ['LOW', 'LOW_PRICE', 'LwPric'],
                'CLOSE': ['CLOSE', 'CLOSE_PRICE', 'LAST', 'ClsPric', 'LastPric'],
                'TOTTRDQTY': ['TOTTRDQTY', 'VOLUME', 'TOTALTRDQTY', 'TradedQty', 'TtlTradgVol'],
                'TOTTRDVAL': ['TOTTRDVAL', 'TURNOVER', 'TOTALTRDVAL', 'TtlTrfVal', 'TtlTradedVal'],
                'TOTALTRADES': ['TOTALTRADES', 'TRADES', 'TotTrdsExctd'],
                'TIMESTAMP': ['TIMESTAMP', 'DATE', 'TradDt', 'BizDt']
            }
            for a in aliases.get(col, [col]):
                if a in cols:
                    return cols[a]
                au = a.upper()
                if au in U:
                    return U[au]
            return None

        # pick columns
        pick = {k: get(k) for k in ['SYMBOL','SERIES','OPEN','HIGH','LOW','CLOSE','TOTTRDQTY','TOTTRDVAL','TOTALTRADES','TIMESTAMP']}
        # create a normalized frame
        out = pd.DataFrame()
        for k, src in pick.items():
            if src is not None and src in df.columns:
                out[k] = df[src]
            else:
                out[k] = np.nan
        # inject source date if missing
        if out['TIMESTAMP'].isna().all():
            fdate = r['file_date']
            out['TIMESTAMP'] = pd.to_datetime(fdate) if fdate else pd.NaT
        else:
            out['TIMESTAMP'] = pd.to_datetime(out['TIMESTAMP'], errors='coerce')
        out['__source_file'] = fp

        if debug_prints < 2:
            nn = out[['SYMBOL','TIMESTAMP']].notna().sum().to_dict()
            print(f"DEBUG eq out rows={len(out)} non-null: {nn} from {fp.split(os.sep)[-1]}")
            debug_prints += 1

        rows.append(out)
        ok += 1

    if not rows:
        print(f"Equity bhav read summary: ok={ok}, failed={fail}")
        return pd.DataFrame()

    eq = pd.concat(rows, ignore_index=True)
    # drop rows without symbol or timestamp
    before = len(eq)
    eq = eq.dropna(subset=['SYMBOL','TIMESTAMP'])
    after = len(eq)
    print(f"Equity bhav read summary: ok={ok}, failed={fail}, rows={len(eq)} (dropped {before-after} missing symbol/date)")
    return eq

In [103]:
equity_bhav_df = read_equity_bhav(files_index)
print("Equity rows:", len(equity_bhav_df))
equity_bhav_df.head(3)

DEBUG eq out rows=3055 non-null: {'SYMBOL': 3055, 'TIMESTAMP': 3055} from BhavCopy_NSE_CM_0_0_0_20250801_F_0000.csv
DEBUG eq out rows=3055 non-null: {'SYMBOL': 3055, 'TIMESTAMP': 3055} from BhavCopy_NSE_CM_0_0_0_20250801_F_0000.csv

DEBUG eq out rows=3055 non-null: {'SYMBOL': 3055, 'TIMESTAMP': 3055} from BhavCopy_NSE_CM_0_0_0_20250801_F_0000.csv
Equity bhav read summary: ok=122, failed=0, rows=384084 (dropped 0 missing symbol/date)
Equity rows: 384084
Equity bhav read summary: ok=122, failed=0, rows=384084 (dropped 0 missing symbol/date)
Equity rows: 384084


,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,TOTTRDQTY,TOTTRDVAL,TOTALTRADES,TIMESTAMP,__source_file
0,SGBJUN28,GB,9863.00,10099.99,9860.0,9939.00,420,4175051.35,NaN,2025-08-01,e:\workspace\nseoptions\data\Full Bhavcopy and...
1,SGBN28VIII,GB,9950.00,9999.00,9925.0,9979.50,31,309284.00,NaN,2025-08-01,e:\workspace\nseoptions\data\Full Bhavcopy and...
2,SGBMAY29I,GB,9929.99,9938.00,9902.0,9932.43,220,2183268.70,NaN,2025-08-01,e:\workspace\nseoptions\data\Full Bhavcopy and...


## 5) Load Security Deliverable Files (Batch)

In [104]:
def read_deliverable(files_df: pd.DataFrame) -> pd.DataFrame:
    de_files = files_df[files_df['file_type'] == 'deliverable']
    if de_files.empty:
        print("No deliverable files found.")
        return pd.DataFrame()

    rows = []
    for _, r in de_files.sort_values('file_date', na_position='last').iterrows():
        fp = r['file_path']
        try:
            if str(fp).lower().endswith('.zip'):
                df = pd.read_csv(fp, compression='zip')
            else:
                df = pd.read_csv(fp, compression='infer')
        except Exception as e:
            warnings.warn(f"Failed to read {fp}: {e}")
            continue

        cols = {c.strip().upper(): c for c in df.columns}
        def pick(*names):
            for n in names:
                if n in cols:
                    return cols[n]
            return None

        out = pd.DataFrame()
        out['SYMBOL'] = df[pick('SYMBOL','SYMB','TICKER')] if pick('SYMBOL','SYMB','TICKER') else np.nan
        # try multiple possible names from NSE deliverable formats
        qty_col = pick('DELIV_QTY','DELIVERY_QTY','DELIV_QTY.','DELIVERABLE QTY','DELIVERABLE_QTY')
        per_col = pick('DELIV_PERC','DELIV_PER','DELIVERY_TO_TRADE_%','DELIV%','DELIV_PERCENT')
        tot_trades_col = pick('TOTAL_TRADES','TRADES')
        date_col = pick('DATE','TIMESTAMP')

        out['DELIV_QTY'] = pd.to_numeric(df[qty_col], errors='coerce') if qty_col else np.nan
        out['DELIV_PERC'] = pd.to_numeric(df[per_col], errors='coerce') if per_col else np.nan
        out['TOTALTRADES'] = pd.to_numeric(df[tot_trades_col], errors='coerce') if tot_trades_col else np.nan
        if date_col:
            out['TIMESTAMP'] = pd.to_datetime(df[date_col], errors='coerce')
        else:
            fdate = r['file_date']
            out['TIMESTAMP'] = pd.to_datetime(fdate) if fdate else pd.NaT
        out['__source_file'] = fp
        rows.append(out)

    if not rows:
        return pd.DataFrame()

    return pd.concat(rows, ignore_index=True)

In [105]:
deliver_df = read_deliverable(files_index)
print("Deliverable rows:", len(deliver_df))
deliver_df.head(3)

No deliverable files found.
Deliverable rows: 0


""


## 6) Load Derivatives Reports (Optional)

In [106]:
def read_derivatives(files_df: pd.DataFrame) -> pd.DataFrame:
    dr_files = files_df[files_df['file_type'] == 'derivatives']
    if dr_files.empty:
        print("No derivatives files found.")
        return pd.DataFrame()

    rows = []
    ok, fail = 0, 0
    for _, r in dr_files.sort_values('file_date', na_position='last').iterrows():
        fp = r['file_path']
        try:
            df = pd.read_csv(fp, compression='infer', low_memory=False)
        except Exception:
            try:
                df = pd.read_csv(fp, compression='zip', low_memory=False)
            except Exception as e:
                warnings.warn(f"Failed to read {fp}: {e}")
                fail += 1
                continue

        cols = {str(c).strip(): str(c).strip() for c in df.columns}
        U = {k.upper(): v for k, v in cols.items()}
        def pick(*names):
            for n in names:
                if n in cols:
                    return cols[n]
                nu = n.upper()
                if nu in U:
                    return U[nu]
            return None

        # Try NSE classic FO schema
        instr_col = pick('INSTRUMENT')
        sym_col = pick('SYMBOL','SYMB','TckrSymb')
        ts_col = pick('TIMESTAMP','DATE','TradDt','BizDt')
        oi_col = pick('OPEN_INT','OPENINTEREST','OI','OpnIntrst')
        chg_oi_col = pick('CHG_IN_OI','CHANGE_IN_OI','CHNG_IN_OI','ChngInOpnIntrst')
        expiry_col = pick('EXPIRY_DT','EXPIRY','XpryDt','FininstrmActlXpryDt')
        series_col = pick('SERIES','SctySrs')
        instr_type_col = pick('INSTRUMENT','FinInstrmTp')

        out = pd.DataFrame()
        # Map instrument: prefer instrument type if present
        if instr_type_col:
            out['INSTRUMENT'] = df[instr_type_col]
        elif instr_col:
            out['INSTRUMENT'] = df[instr_col]
        else:
            out['INSTRUMENT'] = np.nan

        out['SYMBOL'] = df[sym_col] if sym_col else np.nan
        out['TIMESTAMP'] = pd.to_datetime(df[ts_col], errors='coerce') if ts_col else pd.NaT
        out['OPEN_INT'] = pd.to_numeric(df[oi_col], errors='coerce') if oi_col else np.nan
        out['CHG_IN_OI'] = pd.to_numeric(df[chg_oi_col], errors='coerce') if chg_oi_col else np.nan
        out['EXPIRY_DT'] = pd.to_datetime(df[expiry_col], errors='coerce') if expiry_col else pd.NaT
        out['SERIES'] = df[series_col] if series_col else np.nan
        out['__source_file'] = fp
        rows.append(out)
        ok += 1

    if not rows:
        print(f"Derivatives read summary: ok={ok}, failed={fail}")
        return pd.DataFrame()

    dr = pd.concat(rows, ignore_index=True)

    # Normalize: instrument naming from NSE MII FinInstrmTp -> FUTSTK/OPTSTK mapping
    if 'INSTRUMENT' in dr.columns:
        dr['INSTRUMENT'] = dr['INSTRUMENT'].astype(str).str.upper().replace({
            'FUTSTK': 'FUTSTK', 'FUTIDX': 'FUTIDX',
            'OPTSTK': 'OPTSTK', 'OPTIDX': 'OPTIDX',
            'FUT': 'FUTSTK', 'OPT': 'OPTSTK'
        })

    # Filter to futures on stocks and nearest expiry per date
    dr['symbol'] = dr['SYMBOL'].astype(str).str.upper().str.strip()
    dr['date'] = pd.to_datetime(dr['TIMESTAMP'], errors='coerce').dt.date

    # If instrument type absent, infer from columns present
    if 'INSTRUMENT' not in dr or dr['INSTRUMENT'].isna().all():
        dr['INSTRUMENT'] = np.where(dr['CHG_IN_OI'].notna() | dr['OPEN_INT'].notna(), 'FUTSTK', np.nan)

    dr = dr[dr['INSTRUMENT'].astype(str).str.upper().str.contains('FUT')]

    # nearest expiry per symbol-date
    dr['days_to_exp'] = (pd.to_datetime(dr['EXPIRY_DT'], errors='coerce').dt.date - dr['date']).apply(lambda d: d.days if pd.notnull(d) else np.nan)
    dr = dr.sort_values(['symbol','date','days_to_exp']).groupby(['symbol','date'], as_index=False).first()

    print(f"Derivatives read summary: ok={ok}, failed={fail}, rows={len(dr)}")
    return dr

deriv_df = read_derivatives(files_index)
print("Derivatives rows:", len(deriv_df))
deriv_df.head(3)

Derivatives read summary: ok=122, failed=0, rows=0
Derivatives rows: 0


,symbol,date,INSTRUMENT,SYMBOL,TIMESTAMP,OPEN_INT,CHG_IN_OI,EXPIRY_DT,SERIES,__source_file,days_to_exp


## 7) Clean and Normalize Columns

In [107]:
def standardize_equity(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    out = df.copy()
    out.columns = [c.strip().lower() for c in out.columns]
    # unify date
    if 'timestamp' in out.columns:
        out['date'] = pd.to_datetime(out['timestamp'], errors='coerce').dt.date
    elif 'date' in out.columns:
        out['date'] = pd.to_datetime(out['date'], errors='coerce').dt.date
    # unify symbol
    if 'symbol' in out.columns:
        out['symbol'] = out['symbol'].astype(str).str.strip().str.upper()
    # numerics
    for col in ['open','high','low','close','tottrdqty','tottrdval','totaltrades']:
        if col in out.columns:
            out[col] = pd.to_numeric(out[col], errors='coerce')
    out = out.dropna(subset=['symbol','date'])
    out = out.sort_values(['symbol','date']).drop_duplicates(['symbol','date'], keep='last')
    return out

def standardize_deliver(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    out = df.copy()
    out.columns = [c.strip().lower() for c in out.columns]
    # date
    if 'timestamp' in out.columns:
        out['date'] = pd.to_datetime(out['timestamp'], errors='coerce').dt.date
    elif 'date' in out.columns:
        out['date'] = pd.to_datetime(out['date'], errors='coerce').dt.date
    # symbol
    if 'symbol' in out.columns:
        out['symbol'] = out['symbol'].astype(str).str.strip().str.upper()
    # numerics
    for col in ['deliv_qty','deliv_per','deliv_perc','totaltrades']:
        if col in out.columns:
            out[col] = pd.to_numeric(out[col], errors='coerce')
    # unify percentage column name
    if 'deliv_perc' not in out.columns:
        if 'deliv_per' in out.columns:
            out['deliv_perc'] = out['deliv_per']
        else:
            out['deliv_perc'] = np.nan
    out = out.dropna(subset=['symbol','date'])
    out = out.sort_values(['symbol','date'])
    return out

def standardize_deriv(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    out = df.copy()
    out.columns = [c.strip().lower() for c in out.columns]
    if 'timestamp' in out.columns:
        out['date'] = pd.to_datetime(out['timestamp'], errors='coerce').dt.date
    elif 'date' in out.columns:
        out['date'] = pd.to_datetime(out['date'], errors='coerce').dt.date
    if 'symbol' in out.columns:
        out['symbol'] = out['symbol'].astype(str).str.strip().str.upper()
    for col in ['open_int','chg_in_oi']:
        if col in out.columns:
            out[col] = pd.to_numeric(out[col], errors='coerce')
    out = out.dropna(subset=['symbol','date'])
    out = out.sort_values(['symbol','date']).drop_duplicates(['symbol','date'], keep='last')
    return out

# apply standardization
EQ = standardize_equity(equity_bhav_df)
DE = standardize_deliver(deliver_df)
DR = standardize_deriv(deriv_df)

print("EQ shape:", EQ.shape, "DE shape:", DE.shape, "DR shape:", DR.shape)
EQ.head(3)

EQ shape: (191451, 12) DE shape: (0, 0) DR shape: (0, 11)


,symbol,series,open,high,low,close,tottrdqty,tottrdval,totaltrades,timestamp,__source_file,date
48682,0MOFSL26,N1,1093.00,1093.00,1093.00,1093.00,1,1093.00,NaN,2025-08-12,e:\workspace\nseoptions\data\Full Bhavcopy and...,2025-08-12
79613,0MOFSL26,N1,1119.00,1119.00,1119.00,1119.00,2,2238.00,NaN,2025-08-20,e:\workspace\nseoptions\data\Full Bhavcopy and...,2025-08-20
148238,0MOFSL26,N1,1116.38,1116.38,1116.38,1116.38,4,4465.52,NaN,2025-09-05,e:\workspace\nseoptions\data\Full Bhavcopy and...,2025-09-05


## 8) Build Master Equity Dataset

In [108]:
def build_master_equity(eq: pd.DataFrame) -> pd.DataFrame:
    if eq.empty:
        return eq
    out = eq.copy()
    # filter series if available
    if 'series' in out.columns:
        out = out[out['series'].fillna('').str.upper().isin(['EQ','BE'])]
    # computed fields
    for col in ['high','low','close']:
        if col not in out.columns:
            out[col] = np.nan
    out['avg_price'] = (out['high'] + out['low'] + out['close']) / 3.0
    if 'tottrdval' in out.columns:
        out['turnover'] = out['tottrdval']
    out = out.set_index(['symbol','date']).sort_index()
    return out

MASTER_EQ = build_master_equity(EQ)
print("MASTER_EQ index levels:", MASTER_EQ.index.names, "rows:", len(MASTER_EQ))
MASTER_EQ.head(3)

MASTER_EQ index levels: ['symbol', 'date'] rows: 146455


series    open    high     low   close  tottrdqty  \
symbol    date                                                           
20MICRONS 2025-08-01     EQ  234.41  241.03  233.56  235.65     205134   
          2025-08-04     EQ  237.88  246.00  235.65  244.33     258114   
          2025-08-05     EQ  244.33  246.03  234.05  236.15     258339   

                        tottrdval  totaltrades  timestamp  \
symbol    date                                              
20MICRONS 2025-08-01  48729806.54          NaN 2025-08-01   
          2025-08-04  62397303.02          NaN 2025-08-04   
          2025-08-05  61911864.42          NaN 2025-08-05   

                                                          __source_file  \
symbol    date                                                            
20MICRONS 2025-08-01  e:\workspace\nseoptions\data\Full Bhavcopy and...   
          2025-08-04  e:\workspace\nseoptions\data\Full Bhavcopy and...   
          2025-08-05  e:\workspace\nseoptions\data\Full Bhavcopy and...   

                       avg_price     turnover  
symbol    date                                 
20MICRONS 2025-08-01  236.746667  48729806.54  
          2025-08-04  241.993333  62397303.02  
          2025-08-05  238.743333  61911864.42

## 9) Merge Delivery Metrics

In [109]:
def merge_delivery(master: pd.DataFrame, de: pd.DataFrame) -> pd.DataFrame:
    if master.empty or de.empty:
        return master
    d = de[['symbol','date'] + [c for c in ['deliv_qty','deliv_perc','totaltrades'] if c in de.columns]].copy()
    d = d.drop_duplicates(['symbol','date'], keep='last')
    out = master.reset_index().merge(d, on=['symbol','date'], how='left')
    # compute delivery_value if close available
    if 'close' in out.columns and 'deliv_qty' in out.columns:
        out['delivery_value'] = out['close'] * out['deliv_qty']
    # ensure delivery_percent 0..100
    if 'deliv_perc' in out.columns:
        out['delivery_percent'] = out['deliv_perc']
        # if looks like fraction (<=1), convert to percent
        if out['delivery_percent'].dropna().between(0,1).mean() > 0.5:
            out['delivery_percent'] = out['delivery_percent'] * 100.0
    else:
        out['delivery_percent'] = np.nan
    return out.set_index(['symbol','date']).sort_index()

MASTER_EQ = merge_delivery(MASTER_EQ, DE)
MASTER_EQ.head(3)

series    open    high     low   close  tottrdqty  \
symbol    date                                                           
20MICRONS 2025-08-01     EQ  234.41  241.03  233.56  235.65     205134   
          2025-08-04     EQ  237.88  246.00  235.65  244.33     258114   
          2025-08-05     EQ  244.33  246.03  234.05  236.15     258339   

                        tottrdval  totaltrades  timestamp  \
symbol    date                                              
20MICRONS 2025-08-01  48729806.54          NaN 2025-08-01   
          2025-08-04  62397303.02          NaN 2025-08-04   
          2025-08-05  61911864.42          NaN 2025-08-05   

                                                          __source_file  \
symbol    date                                                            
20MICRONS 2025-08-01  e:\workspace\nseoptions\data\Full Bhavcopy and...   
          2025-08-04  e:\workspace\nseoptions\data\Full Bhavcopy and...   
          2025-08-05  e:\workspace\nseoptions\data\Full Bhavcopy and...   

                       avg_price     turnover  
symbol    date                                 
20MICRONS 2025-08-01  236.746667  48729806.54  
          2025-08-04  241.993333  62397303.02  
          2025-08-05  238.743333  61911864.42

## 10) Merge Derivatives Metrics (OI, OI Change)

In [110]:
def merge_derivatives(master: pd.DataFrame, dr: pd.DataFrame) -> pd.DataFrame:
    if master.empty or dr.empty:
        return master
    d = dr[['symbol','date'] + [c for c in ['open_int','chg_in_oi'] if c in dr.columns]].copy()
    d = d.drop_duplicates(['symbol','date'], keep='last')
    out = master.reset_index().merge(d, on=['symbol','date'], how='left')
    return out.set_index(['symbol','date']).sort_index()

MASTER_EQ = merge_derivatives(MASTER_EQ, DR)
MASTER_EQ.head(3)

series    open    high     low   close  tottrdqty  \
symbol    date                                                           
20MICRONS 2025-08-01     EQ  234.41  241.03  233.56  235.65     205134   
          2025-08-04     EQ  237.88  246.00  235.65  244.33     258114   
          2025-08-05     EQ  244.33  246.03  234.05  236.15     258339   

                        tottrdval  totaltrades  timestamp  \
symbol    date                                              
20MICRONS 2025-08-01  48729806.54          NaN 2025-08-01   
          2025-08-04  62397303.02          NaN 2025-08-04   
          2025-08-05  61911864.42          NaN 2025-08-05   

                                                          __source_file  \
symbol    date                                                            
20MICRONS 2025-08-01  e:\workspace\nseoptions\data\Full Bhavcopy and...   
          2025-08-04  e:\workspace\nseoptions\data\Full Bhavcopy and...   
          2025-08-05  e:\workspace\nseoptions\data\Full Bhavcopy and...   

                       avg_price     turnover  
symbol    date                                 
20MICRONS 2025-08-01  236.746667  48729806.54  
          2025-08-04  241.993333  62397303.02  
          2025-08-05  238.743333  61911864.42

## 11) Select Symbol and Slice Time Range

In [111]:
from typing import Optional

def get_symbol_df(master: pd.DataFrame, symbol: str, start: Optional[date]=None, end: Optional[date]=None) -> pd.DataFrame:
    if master.empty:
        return pd.DataFrame()
    symbol = str(symbol).strip().upper()
    df = master.reset_index()
    df = df[df['symbol'] == symbol].copy()
    if start:
        df = df[df['date'] >= start]
    if end:
        df = df[df['date'] <= end]
    df = df.sort_values('date').reset_index(drop=True)
    return df

SYMBOL = default_symbol
symbol_df = get_symbol_df(MASTER_EQ, SYMBOL, start_date, end_date)
print(SYMBOL, "rows:", len(symbol_df))
symbol_df.head(5)

RELIANCE rows: 61


,symbol,date,series,open,high,low,close,tottrdqty,tottrdval,totaltrades,timestamp,__source_file,avg_price,turnover
0,RELIANCE,2025-08-01,EQ,1386.9,1405.9,1384.3,1393.7,10321171,1.442349e+10,NaN,2025-08-01,e:\workspace\nseoptions\data\Full Bhavcopy and...,1394.633333,1.442349e+10
1,RELIANCE,2025-08-04,EQ,1395.1,1416.1,1391.7,1411.5,6312820,8.904908e+09,NaN,2025-08-04,e:\workspace\nseoptions\data\Full Bhavcopy and...,1406.433333,8.904908e+09
2,RELIANCE,2025-08-05,EQ,1411.0,1411.0,1388.1,1391.7,9315972,1.299116e+10,NaN,2025-08-05,e:\workspace\nseoptions\data\Full Bhavcopy and...,1396.933333,1.299116e+10
3,RELIANCE,2025-08-06,EQ,1391.9,1395.9,1383.3,1392.8,5834312,8.113007e+09,NaN,2025-08-06,e:\workspace\nseoptions\data\Full Bhavcopy and...,1390.666667,8.113007e+09
4,RELIANCE,2025-08-07,EQ,1387.7,1393.7,1366.0,1389.4,9976705,1.378305e+10,NaN,2025-08-07,e:\workspace\nseoptions\data\Full Bhavcopy and...,1383.033333,1.378305e+10


## 12) Compute Returns and Indicators

In [112]:
def add_indicators(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    out = df.copy()
    out['close'] = pd.to_numeric(out['close'], errors='coerce')
    out['tottrdqty'] = pd.to_numeric(out.get('tottrdqty', np.nan), errors='coerce')
    out['deliv_qty'] = pd.to_numeric(out.get('deliv_qty', np.nan), errors='coerce')
    out['delivery_percent'] = pd.to_numeric(out.get('delivery_percent', np.nan), errors='coerce')
    out['ret_log'] = np.log(out['close'] / out['close'].shift(1))
    out['vol_20'] = out['ret_log'].rolling(20).std() * np.sqrt(252)
    out['sma20'] = out['close'].rolling(20).mean()
    out['sma50'] = out['close'].rolling(50).mean()
    out['delivery_ratio'] = out['deliv_qty'] / out['tottrdqty']
    return out

symbol_df = add_indicators(symbol_df)
symbol_df.tail(5)

,symbol,date,series,open,high,low,close,tottrdqty,tottrdval,totaltrades,...,__source_file,avg_price,turnover,deliv_qty,delivery_percent,ret_log,vol_20,sma20,sma50,delivery_ratio
56,RELIANCE,2025-10-24,EQ,1450.0,1459.8,1441.0,1451.6,9700837,1.406862e+10,NaN,...,e:\workspace\nseoptions\data\Full Bhavcopy and...,1450.800000,1.406862e+10,NaN,NaN,0.002207,0.171750,1393.895,1390.912,NaN
57,RELIANCE,2025-10-27,EQ,1463.0,1485.0,1458.0,1484.1,14113867,2.089188e+10,NaN,...,e:\workspace\nseoptions\data\Full Bhavcopy and...,1475.700000,2.089188e+10,NaN,NaN,0.022142,0.180857,1399.480,1392.986,NaN
58,RELIANCE,2025-10-28,EQ,1484.1,1492.0,1477.6,1486.9,17631577,2.618225e+10,NaN,...,e:\workspace\nseoptions\data\Full Bhavcopy and...,1485.500000,2.618225e+10,NaN,NaN,0.001885,0.181001,1404.945,1395.072,NaN
59,RELIANCE,2025-10-29,EQ,1490.0,1508.3,1488.1,1504.2,11884745,1.785880e+10,NaN,...,e:\workspace\nseoptions\data\Full Bhavcopy and...,1500.200000,1.785880e+10,NaN,NaN,0.011568,0.180829,1411.515,1397.680,NaN
60,RELIANCE,2025-10-30,EQ,1500.0,1503.1,1484.0,1488.5,9318663,1.390592e+10,NaN,...,e:\workspace\nseoptions\data\Full Bhavcopy and...,1491.866667,1.390592e+10,NaN,NaN,-0.010492,0.184641,1417.740,1399.816,NaN


## 13) Visualize Price and Volume

In [113]:
def plot_price_volume(df: pd.DataFrame, symbol: str):
    if df.empty:
        warnings.warn("No data to plot")
        return None
    t = df.copy()
    t['date'] = pd.to_datetime(t['date'])
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=t['date'], y=t['close'], name='Close', line=dict(color='royalblue')), secondary_y=False)
    if 'sma20' in t:
        fig.add_trace(go.Scatter(x=t['date'], y=t['sma20'], name='SMA20', line=dict(color='orange', width=1)), secondary_y=False)
    if 'sma50' in t:
        fig.add_trace(go.Scatter(x=t['date'], y=t['sma50'], name='SMA50', line=dict(color='green', width=1)), secondary_y=False)

    if 'tottrdqty' in t:
        fig.add_trace(go.Bar(x=t['date'], y=t['tottrdqty'], name='Volume', marker_color='gray', opacity=0.4), secondary_y=True)

    fig.update_layout(title=f"{symbol} - Close with SMA and Volume", template='plotly_white', hovermode='x unified')
    fig.update_xaxes(rangeslider_visible=True)
    fig.update_yaxes(title_text='Price', secondary_y=False)
    fig.update_yaxes(title_text='Volume', secondary_y=True)
    fig.show()
    return fig

fig_price = plot_price_volume(symbol_df, SYMBOL)

## 14) Visualize Candlestick with Delivery Overlay

In [114]:
def plot_candlestick_delivery(df: pd.DataFrame, symbol: str):
    if df.empty:
        return None
    t = df.copy()
    t['date'] = pd.to_datetime(t['date'])
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Candlestick(x=t['date'], open=t['open'], high=t['high'], low=t['low'], close=t['close'], name='OHLC'))

    if 'delivery_percent' in t:
        fig.add_trace(go.Scatter(x=t['date'], y=t['delivery_percent'], name='Delivery %', line=dict(color='purple')), secondary_y=True)
    if 'tottrdqty' in t:
        fig.add_trace(go.Bar(x=t['date'], y=t['tottrdqty'], name='Volume', marker_color='gray', opacity=0.3), secondary_y=True)

    # highlight delivery spikes
    if 'delivery_percent' in t:
        spikes = t[t['delivery_percent'] > 75]
        for _, r in spikes.iterrows():
            fig.add_vrect(x0=r['date']-pd.Timedelta(days=0.5), x1=r['date']+pd.Timedelta(days=0.5), fillcolor='purple', opacity=0.05, line_width=0)

    fig.update_layout(title=f"{symbol} - Candlestick with Delivery%", template='plotly_white', hovermode='x unified')
    fig.update_xaxes(rangeslider_visible=True)
    fig.update_yaxes(title_text='Price', secondary_y=False)
    fig.update_yaxes(title_text='Delivery % / Volume', secondary_y=True)
    fig.show()
    return fig

fig_candle = plot_candlestick_delivery(symbol_df, SYMBOL)

## 15) Visualize OI and OI Change

In [115]:
def plot_oi(df: pd.DataFrame, symbol: str):
    if df.empty or ('open_int' not in df and 'chg_in_oi' not in df):
        print("No OI data available.")
        return None
    t = df.copy()
    t['date'] = pd.to_datetime(t['date'])
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if 'open_int' in t:
        fig.add_trace(go.Bar(x=t['date'], y=t['open_int'], name='Open Interest', marker_color='teal', opacity=0.5), secondary_y=True)
    if 'chg_in_oi' in t:
        fig.add_trace(go.Scatter(x=t['date'], y=t['chg_in_oi'], name='Chg in OI', line=dict(color='firebrick')), secondary_y=True)

    fig.add_trace(go.Scatter(x=t['date'], y=t['close'], name='Close', line=dict(color='royalblue')), secondary_y=False)

    fig.update_layout(title=f"{symbol} - Price vs OI", template='plotly_white', hovermode='x unified')
    fig.update_xaxes(rangeslider_visible=True)
    fig.update_yaxes(title_text='Price', secondary_y=False)
    fig.update_yaxes(title_text='OI / Change', secondary_y=True)
    fig.show()
    return fig

fig_oi = plot_oi(symbol_df, SYMBOL)

No OI data available.


## 16) Interactive Symbol and Date Widgets

In [116]:
if WIDGETS_AVAILABLE:
    symbols = sorted(MASTER_EQ.reset_index()['symbol'].unique().tolist()) if not MASTER_EQ.empty else [default_symbol]
    symbol_dd = widgets.Dropdown(options=symbols, value=default_symbol, description='Symbol:')
    start_dp = widgets.DatePicker(value=start_date, description='Start')
    end_dp = widgets.DatePicker(value=end_date, description='End')
    btn = widgets.Button(description='Update', button_style='primary')

    out = widgets.Output()

    def on_update(_):
        with out:
            clear_output(wait=True)
            sym = symbol_dd.value
            s = get_symbol_df(MASTER_EQ, sym, start_dp.value, end_dp.value)
            s = add_indicators(s)
            print(f"{sym}: {len(s)} rows from {start_dp.value} to {end_dp.value}")
            _ = plot_price_volume(s, sym)
            _ = plot_candlestick_delivery(s, sym)
            _ = plot_oi(s, sym)

    btn.on_click(on_update)
    display(widgets.HBox([symbol_dd, start_dp, end_dp, btn]))
    display(out)
else:
    print("ipywidgets not available; using static default symbol plots above.")

Output()

## 17) Export Charts and CSV Reports

In [117]:
def export_reports(df: pd.DataFrame, symbol: str, start: date, end: date, outdir: Path = REPORTS_DIR):
    if df.empty:
        print("No data to export.")
        return
    outdir.mkdir(parents=True, exist_ok=True)
    span = f"{start.strftime('%Y%m%d')}_{end.strftime('%Y%m%d')}"
    csv_path = outdir / f"{symbol}_{span}.csv"
    df.to_csv(csv_path, index=False)
    print("Saved:", csv_path)

    # Export interactive HTML plots
    f1 = plot_price_volume(df, symbol)
    f2 = plot_candlestick_delivery(df, symbol)
    f3 = plot_oi(df, symbol)
    if f1:
        f1.write_html(outdir / f"{symbol}_{span}_price_volume.html")
    if f2:
        f2.write_html(outdir / f"{symbol}_{span}_candlestick_delivery.html")
    if f3:
        f3.write_html(outdir / f"{symbol}_{span}_oi.html")

# Example export for default symbol
export_reports(symbol_df, SYMBOL, start_date, end_date)

Saved: notebooks\reports\RELIANCE_20250801_20251030.csv


No OI data available.


## 18) Cache and Incremental Refresh

In [118]:
CACHE_DIR = Path('.cache')
CACHE_DIR.mkdir(exist_ok=True)

def df_hash(df: pd.DataFrame, cols: list) -> str:
    h = hashlib.sha256()
    for c in cols:
        if c in df.columns:
            h.update(pd.util.hash_pandas_object(df[c], index=False).values)
    return h.hexdigest()

# Simple placeholders showing where caching would be integrated
print("Cache directory:", CACHE_DIR.resolve())

Cache directory: E:\workspace\nseoptions\notebooks\.cache


## 19) Sanity Checks and Assertions

In [119]:
def sanity_checks(master: pd.DataFrame):
    print("File counts by type:")
    print(files_index.groupby('file_type')['file_path'].count())
    if master.empty:
        print("Master dataset is empty.")
        return
    # expected columns
    expected = {'open','high','low','close','tottrdqty'}
    missing = expected - set(master.columns)
    if missing:
        warnings.warn(f"Missing columns in master: {missing}")
    # date monotonic within symbol
    tmp = master.reset_index().sort_values(['symbol','date'])
    grp = tmp.groupby('symbol')['date'].apply(lambda s: int((s.diff().dt.days.dropna() < 0).sum()))
    bad = grp[grp > 0]
    if len(bad):
        warnings.warn(f"Non-monotonic date sequences for: {bad.index.tolist()}")
    # delivery percent range
    if 'delivery_percent' in master:
        rng_bad = master['delivery_percent'].dropna().pipe(lambda s: ((s < 0) | (s > 100)).sum())
        if rng_bad:
            warnings.warn(f"Out-of-range delivery_percent rows: {rng_bad}")

sanity_checks(MASTER_EQ)
print("Setup complete. Use widgets above or adjust SYMBOL and re-run plotting cells.")

File counts by type:
file_type
derivatives    122
equity_bhav    122
Name: file_path, dtype: int64


AttributeError: Can only use .dt accessor with datetimelike values